## Desafio 2 - Etapa 1 - Web Scrapping

In [1]:
# Importando bibliotecas necessárias para o desafio 2
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

- Caso aconteça algum erro ao carregar o "driver" relacionado à versão do Google Chrome, verificar a versão do navegador instalado e baixar o executável do "driver" referente ao mesmo. Segue o link para download: https://chromedriver.chromium.org/downloads

In [2]:
# Criando variável 'driver' referente ao 'selenium' e abrindo o navegador para os procedimentos automatizados
driver = webdriver.Chrome('chromedriver.exe')

# Criando variável referente ao site
site_estatisticas = 'https://1xbet.whoscored.com/Statistics'

# Acessando o site na página de estatísticas
driver.get(site_estatisticas)
time.sleep(3)

In [3]:
#### TEAM - SUMMARY ####

## FUNÇÃO PARA GERAR TABELA "SUMMARY" ##

# Criando função 'gera_tab_team_summary' para gerar tabelas com os dados da página
def gera_tab_team_summary():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", dentro da guia "Summary".'''
       
    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[2]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # Variável com os elementos para verificar a quantidade de equipes na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # Criando lista das colunas da tabela
    colunas_summary = []

    # 'Looping' para armazenar os nomes das colunas da tabela
    for elem in elementos_tab_th:
        colunas_summary.append(elem.text)

    # Criando variáveis referentes aos elementos com as quantidades de cartões amarelo e vermelho (para tratar a quantidade de cartões separadamente)
    elementos_tab_span_y = conteudo_site.find_all('span', class_='yellow-card-box')
    elementos_tab_span_r = conteudo_site.find_all('span', class_='red-card-box')

    # Criando lista de listas referente aos dados da tabela
    dados_equipes = []

    # Criando variáveis para funcionar dentro do 'Looping'
    qtd_colunas = len(colunas_summary)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados das equipes 'summary'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_equipes = int(elementos_tab_dl[2].b.text.split()[6]) - int(elementos_tab_dl[2].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_equipes):
        equipe = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            equipe.append(elementos_tab_td[idx].text)
            idx += 1
        dados_equipes.append(equipe)

    # Realizando ajuste nos dados obtidos na página referente aos cartões amarelo e vermelho ('Discipline')
    colunas_summary.insert(4, 'Discipline_yellow')
    colunas_summary[5] = 'Discipline_red'

    # 'Looping' para obter os dados dos cartões amarelo e vermelho de cada time disponível na página
    for i, lista in enumerate(dados_equipes):
        lista.pop(4)
        lista.insert(4, elementos_tab_span_y[i].text)
        lista.insert(5, elementos_tab_span_r[i].text)

    # Criando 'DataFrame' da tabela 'Summary'
    tab_summary = pd.DataFrame(dados_equipes, columns=colunas_summary)
    
    return tab_summary

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "SUMMARY" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_team_summary' para gerar tabelas com os dados da página
def gera_paginas_team_summary(botao_submenu='/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/div[2]/dl/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", todas as páginas dentro da guia "Summary" de um \
dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menuse uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Summary"
    btn_summary = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[1]/ul/li[1]/a')
    btn_summary.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[2]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_summary = gera_tab_team_summary()

    # Instanciando botão "next" da página 'Summary'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[2]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Summary'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_team_summary()
        tabela_summary = pd.concat([tabela_summary, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[2]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_summary.reset_index(drop=True, inplace=True)
    
    return tabela_summary

In [4]:
## TABELAS "TEAM STATISTICS - SUMMARY (Overall)" ##
team_sum_overall = gera_paginas_team_summary()
team_sum_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Goals,Shots pg,Discipline_yellow,Discipline_red,Possession%,Pass%,AerialsWon,Rating
0,1. Manchester City,Premier League,68,18.5,34,1,68.0,90.0,11.9,7.10
1,2. Liverpool,Premier League,73,19.5,34,1,62.5,84.4,15.8,7.08
2,3. Bayern Munich,Bundesliga,77,20.4,22,1,63.8,85.6,13,7.02
3,4. Chelsea,Premier League,56,14.4,44,1,59.5,86.3,14.7,6.95
4,5. Real Madrid,LaLiga,56,17.4,50,0,61.4,89.1,10.6,6.90


In [5]:
## TABELAS "TEAM STATISTICS - SUMMARY (Home)" ##
team_sum_home = gera_paginas_team_summary('/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/div[2]/dl/dd[2]/a')
team_sum_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Goals,Shots pg,Discipline_yellow,Discipline_red,Possession%,Pass%,AerialsWon,Rating
0,1. Liverpool,Premier League,37,21.5,15,0,64.1,85.1,17.4,7.17
1,2. Manchester City,Premier League,40,18.9,13,1,69.6,90.7,11.9,7.14
2,3. Bayern Munich,Bundesliga,38,20.7,12,0,65.7,86.4,12,7.03
3,4. Paris Saint-Germain,Ligue 1,31,15.7,20,1,62.4,91.0,7.9,6.98
4,5. Real Madrid,LaLiga,32,18.9,24,0,63.2,89.6,11,6.95


In [6]:
## TABELAS "TEAM STATISTICS - SUMMARY (Away)" ##
team_sum_away = gera_paginas_team_summary('/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/div[2]/dl/dd[3]/a')
team_sum_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Goals,Shots pg,Discipline_yellow,Discipline_red,Possession%,Pass%,AerialsWon,Rating
0,1. Manchester City,Premier League,28,18,21,0,66.3,89.2,11.9,7.07
1,2. Bayern Munich,Bundesliga,39,20.2,10,1,61.9,84.6,14,7.00
2,3. Liverpool,Premier League,36,17.5,19,1,61.0,83.6,14.3,6.98
3,4. Chelsea,Premier League,29,12.5,25,1,58.8,86.2,14.8,6.96
4,5. Real Madrid,LaLiga,24,15.8,26,0,59.5,88.4,10.2,6.85


In [7]:
#### TEAM - DEFENSIVE ####

## FUNÇÃO PARA GERAR TABELA "DEFENSIVE" ##

# Criando função 'gera_tab_team_defensive' para gerar tabelas com os dados da página na guia 'Defensive'
def gera_tab_team_defensive():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", dentro da guia "Defensive".'''
       
    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[4]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # Variável com os elementos para verificar a quantidade de equipes na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # Criando lista das colunas da tabela ''Defensive'
    colunas_defensive = []

    # 'Looping' para armazenar os nomes das colunas da tabela
    for elem in elementos_tab_th:
        colunas_defensive.append(elem.text)

    # Criando lista de listas referente aos dados da tabela
    dados_equipes = []
    qtd_colunas = len(colunas_defensive)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados das equipes 'defensive'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_equipes = int(elementos_tab_dl[2].b.text.split()[6]) - int(elementos_tab_dl[2].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_equipes):
        equipe = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            equipe.append(elementos_tab_td[idx].text)
            idx += 1
        dados_equipes.append(equipe)

    # Criando 'DataFrame' da tabela 'Defensive'
    tab_defensive = pd.DataFrame(dados_equipes, columns=colunas_defensive)

    return tab_defensive

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "DEFENSIVE" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_team_defensive' para gerar tabelas com os dados da página
def gera_paginas_team_defensive(botao_submenu='/html/body/div[4]/div[3]/div[3]/div[4]/div[1]/div[2]/dl/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", todas as páginas dentro da guia "Defensive" de um \
dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menuse uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Defensive"
    btn_defensive = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[1]/ul/li[2]/a')
    btn_defensive.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[4]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_defensive = gera_tab_team_defensive()

    # Instanciando botão "next" da página 'Defensive'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[4]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Defensive'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_team_defensive()
        tabela_defensive = pd.concat([tabela_defensive, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[4]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_defensive.reset_index(drop=True, inplace=True)

    return tabela_defensive

In [8]:
## TABELAS "TEAM STATISTICS - DEFENSIVE (Overall)" ##
team_def_overall = gera_paginas_team_defensive()
team_def_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Shots pg,Tackles pg,Interceptions pg,Fouls pg,Offsides pg,Rating
0,1. Monaco,Ligue 1,10,20.1,10.7,13.1,2.5,6.66
1,2. Leeds,Premier League,15.4,19.5,10.5,11.8,1.8,6.51
2,3. Everton,Premier League,13.4,19.3,9.9,9.6,1.6,6.57
3,4. Metz,Ligue 1,13.9,18.9,12.9,13.3,1.2,6.54
4,5. Saint-Etienne,Ligue 1,11.9,18.7,12.5,12,1.4,6.50


In [9]:
## TABELAS "TEAM STATISTICS - DEFENSIVE (Home)" ##
team_def_home = gera_paginas_team_defensive('/html/body/div[4]/div[3]/div[3]/div[4]/div[1]/div[2]/dl/dd[2]/a')
team_def_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Shots pg,Tackles pg,Interceptions pg,Fouls pg,Offsides pg,Rating
0,1. Celta Vigo,LaLiga,9.9,20.1,8.8,13.1,2.4,6.60
1,2. Monaco,Ligue 1,9.5,20.1,10.7,13.3,2.6,6.75
2,3. Lorient,Ligue 1,10.5,18.8,11.4,12.1,1.6,6.58
3,4. Brest,Ligue 1,10.6,18.8,11.8,11.1,1.2,6.73
4,5. Leeds,Premier League,14,18.7,10.3,11.8,1.9,6.57


In [10]:
## TABELAS "TEAM STATISTICS - DEFENSIVE (Away)" ##
team_def_away = gera_paginas_team_defensive('/html/body/div[4]/div[3]/div[3]/div[4]/div[1]/div[2]/dl/dd[3]/a')
team_def_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Shots pg,Tackles pg,Interceptions pg,Fouls pg,Offsides pg,Rating
0,1. Metz,Ligue 1,13.5,21,13.5,13.4,1.1,6.57
1,2. Leeds,Premier League,16.9,20.3,10.7,11.9,1.6,6.45
2,3. Monaco,Ligue 1,10.5,20.1,10.8,12.8,2.3,6.56
3,4. Everton,Premier League,15.3,19.9,9.2,10.6,1.4,6.46
4,5. Saint-Etienne,Ligue 1,13.2,19.9,11.8,13.4,1.1,6.43


In [11]:
#### TEAM - OFFENSIVE ####

## FUNÇÃO PARA GERAR TABELA "OFFENSIVE" ##

# Criando função 'gera_tab_team_offensive' para gerar tabelas com os dados da página na guia 'Offensive'
def gera_tab_team_offensive():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", dentro da guia "Offensive".'''

    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[3]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # Variável com os elementos para verificar a quantidade de equipes na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # Criando lista das colunas da tabela 'Offensive'
    colunas_offensive = []

    # 'Looping' para armazenar os nomes das colunas da tabela 'Offensive'
    for elem in elementos_tab_th:
        colunas_offensive.append(elem.text)

    # Criando lista de listas referente aos dados da tabela
    dados_equipes = []
    qtd_colunas = len(colunas_offensive)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados das equipes 'offensive'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_equipes = int(elementos_tab_dl[2].b.text.split()[6]) - int(elementos_tab_dl[2].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_equipes):
        equipe = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            equipe.append(elementos_tab_td[idx].text)
            idx += 1
        dados_equipes.append(equipe)

    # Criando 'DataFrame' da tabela 'Defensive'
    tab_offensive = pd.DataFrame(dados_equipes, columns=colunas_offensive)
    
    return tab_offensive

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "OFFENSIVE" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_team_offensive' para gerar tabelas com os dados da página
def gera_paginas_team_offensive(botao_submenu='/html/body/div[4]/div[3]/div[3]/div[3]/div[1]/div[2]/dl/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", todas as páginas dentro da guia "Offensive" de um \
dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menuse uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Offensive"
    btn_offensive = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[1]/ul/li[3]/a')
    btn_offensive.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[3]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_offensive = gera_tab_team_offensive()

    # Instanciando botão "next" da página 'Offensive'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[3]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Offensive'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_team_offensive()
        tabela_offensive = pd.concat([tabela_offensive, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[3]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_offensive.reset_index(drop=True, inplace=True)

    return tabela_offensive

In [12]:
## TABELAS "TEAM STATISTICS - OFFENSIVE (Overall)" ##
team_off_overall = gera_paginas_team_offensive()
team_off_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Shots pg,Shots OT pg,Dribbles pg,Fouled pg,Rating
0,1. Bayern Munich,Bundesliga,20.4,8.3,15.3,10.4,7.02
1,2. Paris Saint-Germain,Ligue 1,14.6,5.1,12.9,10.8,6.88
2,3. Lyon,Ligue 1,14,5.3,12.8,10.9,6.75
3,4. Barcelona,LaLiga,13.1,4.8,12.6,14.5,6.76
4,5. Bayer Leverkusen,Bundesliga,13.6,6.1,12.5,9.7,6.79


In [13]:
## TABELAS "TEAM STATISTICS - OFFENSIVE (Home)" ##
team_off_home = gera_paginas_team_offensive('/html/body/div[4]/div[3]/div[3]/div[3]/div[1]/div[2]/dl/dd[2]/a')
team_off_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Shots pg,Shots OT pg,Dribbles pg,Fouled pg,Rating
0,1. Bayern Munich,Bundesliga,20.7,8.3,16.1,10.5,7.03
1,2. Bayer Leverkusen,Bundesliga,15.1,6.8,14.6,9.3,6.86
2,3. Paris Saint-Germain,Ligue 1,15.7,5.5,13.8,11.5,6.98
3,4. Barcelona,LaLiga,14.2,5.8,13.6,14.1,6.83
4,5. AC Milan,Serie A,16,4.4,13.1,14.4,6.77


In [14]:
## TABELAS "TEAM STATISTICS - OFFENSIVE (Away)" ##
team_off_away = gera_paginas_team_offensive('/html/body/div[4]/div[3]/div[3]/div[3]/div[1]/div[2]/dl/dd[3]/a')
team_off_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Shots pg,Shots OT pg,Dribbles pg,Fouled pg,Rating
0,1. Bayern Munich,Bundesliga,20.2,8.3,14.6,10.3,7.00
1,2. Real Madrid,LaLiga,15.8,6.2,12.8,13.8,6.85
2,3. Lyon,Ligue 1,13.2,4.8,12.8,10.1,6.67
3,4. Paris Saint-Germain,Ligue 1,13.5,4.8,11.9,10.1,6.78
4,5. Manchester City,Premier League,18,6.6,11.9,9.4,7.07


In [15]:
#### TEAM - DETAILED ####

## FUNÇÃO PARA GERAR TABELA "DETAILED" ##

# Criando função 'gera_tab_team_detailed' para gerar tabelas com os dados da página na guia 'Detailed'
def gera_tab_team_detailed():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", dentro da guia "Detailed".'''

    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[5]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # Variável com os elementos para verificar a quantidade de equipes na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # Criando lista das colunas da tabela 'Detailed'
    colunas_detailed = []

    # 'Looping' para armazenar os nomes das colunas da tabela 'Detailed'
    for elem in elementos_tab_th:
        colunas_detailed.append(elem.text)

    # Criando lista de listas referente aos dados da tabela
    dados_equipes = []
    qtd_colunas = len(colunas_detailed)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados das equipes 'detailed'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_equipes = int(elementos_tab_dl[4].b.text.split()[6]) - int(elementos_tab_dl[4].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_equipes):
        equipe = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            equipe.append(elementos_tab_td[idx].text)
            idx += 1
        dados_equipes.append(equipe)

    # Criando 'DataFrame' da tabela 'Defensive'
    tab_detailed = pd.DataFrame(dados_equipes, columns=colunas_detailed)
    
    return tab_detailed

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "DETAILED" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_team_detailed' para gerar tabelas com os dados da página
def gera_paginas_team_detailed(botao_submenu='/html/body/div[4]/div[3]/div[3]/div[5]/div[1]/div/div[4]/div[1]/dl/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Team Statistics", todas as páginas dentro da guia "Detailed" de um \
dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menuse uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Detailed"
    btn_detailed = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[1]/ul/li[4]/a')
    btn_detailed.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[5]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_detailed = gera_tab_team_detailed()

    # Instanciando botão "next" da página 'Detailed'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[5]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Detailed'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_team_detailed()
        tabela_detailed = pd.concat([tabela_detailed, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[3]/div[5]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_detailed.reset_index(drop=True, inplace=True)

    return tabela_detailed

In [16]:
## TABELAS "TEAM STATISTICS - DETAILED (Overall)" ##
team_det_overall = gera_paginas_team_detailed()
team_det_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,1. Manchester City,Premier League,18.5,5.5,1.9,11.1,7.10
1,2. Liverpool,Premier League,19.5,6.2,1.9,11.4,7.08
2,3. Bayern Munich,Bundesliga,20.4,5.9,2,12.5,7.02
3,4. Chelsea,Premier League,14.4,5.1,1,8.3,6.95
4,5. Real Madrid,LaLiga,17.4,6.7,1.2,9.5,6.90


In [17]:
## TABELAS "TEAM STATISTICS - DETAILED (Home)" ##
team_det_home = gera_paginas_team_detailed('/html/body/div[4]/div[3]/div[3]/div[5]/div[1]/div/div[4]/div[1]/dl/dd[2]/a')
team_det_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,1. Liverpool,Premier League,21.5,6.9,2,12.6,7.17
1,2. Manchester City,Premier League,18.9,6,1.7,11.2,7.14
2,3. Bayern Munich,Bundesliga,20.7,5.8,2.2,12.8,7.03
3,4. Paris Saint-Germain,Ligue 1,15.7,5.3,1.5,8.9,6.98
4,5. Real Madrid,LaLiga,18.9,7.7,1.3,9.9,6.95


In [18]:
## TABELAS "TEAM STATISTICS - DETAILED (Away)" ##
team_det_away = gera_paginas_team_detailed('/html/body/div[4]/div[3]/div[3]/div[5]/div[1]/div/div[4]/div[1]/dl/dd[3]/a')
team_det_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5


,Team,Tournament,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,1. Manchester City,Premier League,18,5,2,11,7.07
1,2. Bayern Munich,Bundesliga,20.2,6.1,1.8,12.2,7.00
2,3. Liverpool,Premier League,17.5,5.5,1.8,10.2,6.98
3,4. Chelsea,Premier League,12.5,4.4,1,7.1,6.96
4,5. Real Madrid,LaLiga,15.8,5.7,1.1,9,6.85


In [19]:
#### PLAYER - SUMMARY ####

## FUNÇÃO PARA GERAR TABELA "SUMMARY - PLAYER" ##

# Criando função 'gera_tab_player_summary' para gerar tabelas com os dados da página
def gera_tab_player_summary():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", dentro da guia "Summary".'''

    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[1]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # # Variável com os elementos para verificar a quantidade de jogadores na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # # Criando lista das colunas da tabela
    colunas_summary = []

    # # 'Looping' para armazenar os nomes das colunas da tabela
    for elem in elementos_tab_th:
        colunas_summary.append(elem.text)
    # Renomeando a primeira coluna que será excluída futuramente (antigo nome = 'Player', o mesmo nome da segunda coluna), os dados delas são idênticos ao da segunda,
    # mas com um algarismo numérico junto com o primeiro nome de cada jogador
    colunas_summary[0] = 'Remover'

    # # Criando lista de listas referente aos dados da tabela
    dados_jogadores = []

    # Criando variáveis para funcionar dentro do 'Looping'
    qtd_colunas = len(colunas_summary)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados dos jogadores 'summary'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_jogadores = int(elementos_tab_dl[3].b.text.split()[6]) - int(elementos_tab_dl[3].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_jogadores):
        jogador = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            jogador.append(elementos_tab_td[idx].text.strip())
            idx += 1
        dados_jogadores.append(jogador)

    # Criando 'DataFrame' da tabela 'Summary'
    tab_summary = pd.DataFrame(dados_jogadores, columns=colunas_summary)

    return tab_summary.drop(columns='Remover')

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "SUMMARY - PLAYER" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_player_summary' para gerar tabelas com os dados da página
def gera_paginas_player_summary(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[1]/div[1]/div[2]/dl[1]/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", todas as páginas dentro da guia "Summary" de \
um dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menus e uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Summary"
    btn_summary = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/ul/li[1]/a')
    btn_summary.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[1]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_player_summary = gera_tab_player_summary()

    # Instanciando botão "next" da página 'Summary'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[1]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Summary'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_player_summary()
        tabela_player_summary = pd.concat([tabela_player_summary, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[1]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_player_summary.reset_index(drop=True, inplace=True)

    return tabela_player_summary

In [20]:
## TABELAS "PLAYER STATISTICS - SUMMARY (Overall)" ##
player_sum_overall = gera_paginas_player_summary(qtd_paginas=8)
player_sum_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,29,1,2,-,5,77.1,1.7,7,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",23(1),2042,14,10,7,-,4,82.3,0.1,8,7.89
2,"Erling Haaland Borussia Dortmund, 21, FW",13(1),1145,16,5,3,-,4.1,69.9,1.9,4,7.88
3,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,20,10,1,-,4.3,80.2,0.2,6,7.77
4,"Karim Benzema Real Madrid, 34, FW",24,1974,20,10,-,-,3.8,85.6,0.6,9,7.71


In [21]:
## TABELAS "PLAYER STATISTICS - SUMMARY (Home)" ##
player_sum_home = gera_paginas_player_summary(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[1]/div[1]/div[2]/dl[1]/dd[2]/a', qtd_paginas=8)
player_sum_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,"Erling Haaland Borussia Dortmund, 21, FW",7,615,12,2,1,-,4.7,69.7,1.9,3,8.48
1,"Kylian Mbappé PSG, 23, AM(LR),FW",12,1044,9,6,3,-,4.3,81.4,0.1,7,8.29
2,"Dusan Vlahovic Fiorentina, 22, FW",10,898,12,2,2,-,4.6,72.3,1.5,4,8.02
3,"Christopher Nkunku RBL, 24, M(CLR),FW",12(1),1081,8,7,-,-,2.8,82.3,0.8,5,7.93
4,"Robert Lewandowski Bayern, 33, FW",13,1148,12,1,1,-,5.2,81.3,1.6,3,7.90


In [22]:
## TABELAS "PLAYER STATISTICS - SUMMARY (Away)" ##
player_sum_away = gera_paginas_player_summary(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[1]/div[1]/div[2]/dl[1]/dd[3]/a', qtd_paginas=8)
player_sum_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Rating
0,"Robert Lewandowski Bayern, 33, FW",13,1113,17,-,1,-,4.9,73.3,1.7,4,8.15
1,"Joshua Kimmich Bayern, 27, D(CR),DMC,M(R)",10,900,1,6,1,-,1.1,86.7,0.5,3,7.97
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",13,1145,11,6,-,-,3.9,79.5,0.3,2,7.75
3,"Phil Foden Man City, 21, AM(CLR),FW",9,783,6,2,1,-,2.8,90.1,0.1,1,7.71
4,"Zlatan Ibrahimovic AC Milan, 40, FW",6(1),461,7,1,1,-,3.7,73.8,1.4,2,7.70


In [23]:
## Opções de filtragem de jogadores - Summary (Remover comentário do código referente à opção de filtro de jogadores desejada e comentar o que não vai usar) ##
## OBS: caso queira obter os dados dos jogadores na guia Summary alterando o filtro 'Default', após executar essa célula, executar as três células acima novamente

# Minumum apps (Default)
btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[1]/div[1]/div[2]/dl[2]/dd[1]/a') 

# All players
# btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[1]/div[1]/div[2]/dl[2]/dd[2]/a')

btn_filtro.click()
time.sleep(3)

In [24]:
#### PLAYER - DEFENSIVE ####

## FUNÇÃO PARA GERAR TABELA "DEFENSIVE - PLAYER" ##

# Criando função 'gera_tab_player_defensive' para gerar tabelas com os dados da página
def gera_tab_player_defensive():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", dentro da guia "Defensive".'''

    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[2]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # # Variável com os elementos para verificar a quantidade de jogadores na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # # Criando lista das colunas da tabela
    colunas_defensive = []

    # # 'Looping' para armazenar os nomes das colunas da tabela
    for elem in elementos_tab_th:
        colunas_defensive.append(elem.text)
    # Renomeando a primeira coluna que será excluída futuramente (antigo nome = 'Player', o mesmo nome da segunda coluna), os dados delas são idênticos ao da segunda,
    # mas com um algarismo numérico junto com o primeiro nome de cada jogador
    colunas_defensive[0] = 'Remover'

    # # Criando lista de listas referente aos dados da tabela
    dados_jogadores = []

    # Criando variáveis para funcionar dentro do 'Looping'
    qtd_colunas = len(colunas_defensive)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados dos jogadores 'defensive'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_jogadores = int(elementos_tab_dl[3].b.text.split()[6]) - int(elementos_tab_dl[3].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_jogadores):
        jogador = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            jogador.append(elementos_tab_td[idx].text.strip())
            idx += 1
        dados_jogadores.append(jogador)

    # Criando 'DataFrame' da tabela 'defensive'
    tab_defensive = pd.DataFrame(dados_jogadores, columns=colunas_defensive)

    return tab_defensive.drop(columns='Remover')

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "DEFENSIVE - PLAYER" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_player_defensive' para gerar tabelas com os dados da página
def gera_paginas_player_defensive(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[2]/div[1]/div[2]/dl[1]/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", todas as páginas dentro da guia "Defensive" de \
um dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menus e uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Defensive"
    btn_defensive = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/ul/li[2]/a')
    btn_defensive.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[2]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_player_defensive = gera_tab_player_defensive()

    # Instanciando botão "next" da página 'Defensive'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[2]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Defensive'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_player_defensive()
        tabela_player_defensive = pd.concat([tabela_player_defensive, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[2]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_player_defensive.reset_index(drop=True, inplace=True)

    return tabela_player_defensive

In [25]:
## TABELAS "PLAYER STATISTICS - DEFENSIVE (Overall)" ##
player_def_overall = gera_paginas_player_defensive(qtd_paginas=8)
player_def_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,0.3,0.1,0.6,0.1,0.3,0.3,0.1,-,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",23(1),2042,0.3,0.2,1,-,0.5,0.2,-,-,7.89
2,"Erling Haaland Borussia Dortmund, 21, FW",13(1),1145,-,0.1,0.7,0.1,1.3,0.1,0.1,-,7.88
3,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,0.4,0.2,0.3,-,0.2,0.4,-,-,7.77
4,"Karim Benzema Real Madrid, 34, FW",24,1974,0.5,0.2,0.5,-,0.4,0.3,-,-,7.71


In [26]:
## TABELAS "PLAYER STATISTICS - DEFENSIVE (Home)" ##
player_def_home = gera_paginas_player_defensive(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[2]/div[1]/div[2]/dl[1]/dd[2]/a', qtd_paginas=8)
player_def_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"Erling Haaland Borussia Dortmund, 21, FW",7,615,-,-,1,0.1,1.6,0.1,0.1,-,8.48
1,"Kylian Mbappé PSG, 23, AM(LR),FW",12,1044,0.3,0.3,1.1,-,0.4,0.3,-,-,8.29
2,"Dusan Vlahovic Fiorentina, 22, FW",10,898,0.2,-,0.3,-,0.9,0.1,-,-,8.02
3,"Christopher Nkunku RBL, 24, M(CLR),FW",12(1),1081,1,0.5,0.9,-,0.3,0.7,0.1,-,7.93
4,"Robert Lewandowski Bayern, 33, FW",13,1148,0.2,0.1,0.7,-,0.2,0.2,0.1,-,7.90


In [27]:
## TABELAS "PLAYER STATISTICS - DEFENSIVE (Away)" ##
player_def_away = gera_paginas_player_defensive(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[2]/div[1]/div[2]/dl[1]/dd[3]/a', qtd_paginas=8)
player_def_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"Robert Lewandowski Bayern, 33, FW",13,1113,0.4,0.2,0.5,0.2,0.4,0.4,0.1,-,8.15
1,"Joshua Kimmich Bayern, 27, D(CR),DMC,M(R)",10,900,1.2,2.1,0.7,0.1,0.9,1.6,0.4,-,7.97
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",13,1145,0.4,0.4,0.2,-,0.1,0.5,-,-,7.75
3,"Phil Foden Man City, 21, AM(CLR),FW",9,783,0.6,0.6,0.8,-,0.1,0.4,-,-,7.71
4,"Zlatan Ibrahimovic AC Milan, 40, FW",6(1),461,-,0.1,0.7,-,0.6,0.3,-,1,7.70


In [28]:
## Opção de filtragem de jogadores - Defensive (Remover comentário do código referente à opção de filtro de jogadores desejada e comentar o que não vai usar) ##
## OBS: caso queira obter os dados dos jogadores na guia Defensive alterando o filtro 'Default', após executar essa célula, executar as três células acima novamente

# Minumum apps (Default)
btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[2]/div[1]/div[2]/dl[2]/dd[1]/a') 

# All players
# btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[2]/div[1]/div[2]/dl[2]/dd[2]/a')

btn_filtro.click()
time.sleep(3)

In [29]:
#### PLAYER - OFFENSIVE ####

## FUNÇÃO PARA GERAR TABELA "OFFENSIVE - PLAYER" ##

# Criando função 'gera_tab_player_offensive' para gerar tabelas com os dados da página
def gera_tab_player_offensive():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", dentro da guia "Offensive".'''

    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[3]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # # Variável com os elementos para verificar a quantidade de jogadores na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # # Criando lista das colunas da tabela
    colunas_offensive = []

    # # 'Looping' para armazenar os nomes das colunas da tabela
    for elem in elementos_tab_th:
        colunas_offensive.append(elem.text)
    # Renomeando a primeira coluna que será excluída futuramente (antigo nome = 'Player', o mesmo nome da segunda coluna), os dados delas são idênticos ao da segunda,
    # mas com um algarismo numérico junto com o primeiro nome de cada jogador
    colunas_offensive[0] = 'Remover'

    # # Criando lista de listas referente aos dados da tabela
    dados_jogadores = []

    # Criando variáveis para funcionar dentro do 'Looping'
    qtd_colunas = len(colunas_offensive)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados dos jogadores 'offensive'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_jogadores = int(elementos_tab_dl[3].b.text.split()[6]) - int(elementos_tab_dl[3].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_jogadores):
        jogador = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            jogador.append(elementos_tab_td[idx].text.strip())
            idx += 1
        dados_jogadores.append(jogador)

    # Criando 'DataFrame' da tabela 'offensive'
    tab_offensive = pd.DataFrame(dados_jogadores, columns=colunas_offensive)

    return tab_offensive.drop(columns='Remover')

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "OFFENSIVE - PLAYER" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_player_offensive' para gerar tabelas com os dados da página
def gera_paginas_player_offensive(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[3]/div[1]/div[2]/dl[1]/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", todas as páginas dentro da guia "Offensive" de \
um dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menus e uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Offensive"
    btn_offensive = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/ul/li[3]/a')
    btn_offensive.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[3]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_player_offensive = gera_tab_player_offensive()

    # Instanciando botão "next" da página 'Offensive'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[3]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Offensive'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_player_offensive()
        tabela_player_offensive = pd.concat([tabela_player_offensive, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[3]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_player_offensive.reset_index(drop=True, inplace=True)

    return tabela_player_offensive

In [30]:
## TABELAS "PLAYER STATISTICS - OFFENSIVE (Overall)" ##
player_off_overall = gera_paginas_player_offensive(qtd_paginas=8)
player_off_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,29,1,5,1.2,1.5,1.1,0.7,1.3,2.4,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",23(1),2042,14,10,4,2,3.3,1.5,1,1.9,3,7.89
2,"Erling Haaland Borussia Dortmund, 21, FW",13(1),1145,16,5,4.1,0.9,0.7,0.7,0.5,0.6,2.9,7.88
3,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,20,10,4.3,2,1.8,0.6,0.5,2,3,7.77
4,"Karim Benzema Real Madrid, 34, FW",24,1974,20,10,3.8,1.9,1,0.3,1.1,0.7,1.3,7.71


In [31]:
## TABELAS "PLAYER STATISTICS - OFFENSIVE (Home)" ##
player_off_home = gera_paginas_player_offensive(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[3]/div[1]/div[2]/dl[1]/dd[2]/a', qtd_paginas=8)
player_off_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,"Erling Haaland Borussia Dortmund, 21, FW",7,615,12,2,4.7,1,1,0.7,0.7,0.9,2.7,8.48
1,"Kylian Mbappé PSG, 23, AM(LR),FW",12,1044,9,6,4.3,2.3,4.3,1.4,1.1,2.1,3.3,8.29
2,"Dusan Vlahovic Fiorentina, 22, FW",10,898,12,2,4.6,1.3,0.8,2.5,0.9,2.3,4,8.02
3,"Christopher Nkunku RBL, 24, M(CLR),FW",12(1),1081,8,7,2.8,1.9,2.4,1.2,0.5,1.3,3,7.93
4,"Robert Lewandowski Bayern, 33, FW",13,1148,12,1,5.2,1.3,1.5,1,0.6,1.6,2.3,7.90


In [32]:
## TABELAS "PLAYER STATISTICS - OFFENSIVE (Away)" ##
player_off_away = gera_paginas_player_offensive(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[3]/div[1]/div[2]/dl[1]/dd[3]/a', qtd_paginas=8)
player_off_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating
0,"Robert Lewandowski Bayern, 33, FW",13,1113,17,-,4.9,1.2,1.5,1.2,0.7,1.1,2.5,8.15
1,"Joshua Kimmich Bayern, 27, D(CR),DMC,M(R)",10,900,1,6,1.1,4,1,2.5,-,0.9,1.8,7.97
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",13,1145,11,6,3.9,2.1,1.5,0.5,0.3,1.5,3.2,7.75
3,"Phil Foden Man City, 21, AM(CLR),FW",9,783,6,2,2.8,1.8,1.2,1.1,0.9,1.3,2.2,7.71
4,"Zlatan Ibrahimovic AC Milan, 40, FW",6(1),461,7,1,3.7,1.3,0.4,0.7,1.3,1,1.1,7.70


In [33]:
## Opção de filtragem de jogadores - Offensive (Remover comentário do código referente à opção de filtro de jogadores desejada e comentar o que não vai usar) ##
## OBS: caso queira obter os dados dos jogadores na guia Offensive alterando o filtro 'Default', após executar essa célula, executar as três células acima novamente

# Minumum apps (Default)
btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[3]/div[1]/div[2]/dl[2]/dd[1]/a')

# All players
# btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[3]/div[1]/div[2]/dl[2]/dd[2]/a')

btn_filtro.click()
time.sleep(3)

In [34]:
#### PLAYER - PASSING ####

## FUNÇÃO PARA GERAR TABELA "PASSING - PLAYER" ##

# Criando função 'gera_tab_player_passing' para gerar tabelas com os dados da página
def gera_tab_player_passing():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", dentro da guia "Passing".'''

    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[4]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # # Variável com os elementos para verificar a quantidade de jogadores na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # # Criando lista das colunas da tabela
    colunas_passing = []

    # # 'Looping' para armazenar os nomes das colunas da tabela
    for elem in elementos_tab_th:
        colunas_passing.append(elem.text)
    # Renomeando a primeira coluna que será excluída futuramente (antigo nome = 'Player', o mesmo nome da segunda coluna), os dados delas são idênticos ao da segunda,
    # mas com um algarismo numérico junto com o primeiro nome de cada jogador
    colunas_passing[0] = 'Remover'

    # # Criando lista de listas referente aos dados da tabela
    dados_jogadores = []

    # Criando variáveis para funcionar dentro do 'Looping'
    qtd_colunas = len(colunas_passing)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados dos jogadores 'passing'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_jogadores = int(elementos_tab_dl[3].b.text.split()[6]) - int(elementos_tab_dl[3].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_jogadores):
        jogador = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            jogador.append(elementos_tab_td[idx].text.strip())
            idx += 1
        dados_jogadores.append(jogador)

    # Criando 'DataFrame' da tabela 'passing'
    tab_passing = pd.DataFrame(dados_jogadores, columns=colunas_passing)

    return tab_passing.drop(columns='Remover')

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "PASSING - PLAYER" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_player_passing' para gerar tabelas com os dados da página
def gera_paginas_player_passing(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[4]/div[1]/div[2]/dl[1]/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", todas as páginas dentro da guia "Passing" de \
um dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menus e uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Passing"
    btn_passing = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/ul/li[4]/a')
    btn_passing.click()
    time.sleep(2)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(2)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[4]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_player_passing = gera_tab_player_passing()

    # Instanciando botão "next" da página 'Passing'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[4]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Passing'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(3)
        tabela_temp = gera_tab_player_passing()
        tabela_player_passing = pd.concat([tabela_player_passing, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[4]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_player_passing.reset_index(drop=True, inplace=True)

    return tabela_player_passing

In [35]:
## TABELAS "PLAYER STATISTICS - PASSING (Overall)" ##
player_pas_overall = gera_paginas_player_passing(qtd_paginas=8)
player_pas_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,"Robert Lewandowski Bayern, 33, FW",26,2261,1,1.2,21.5,77.1,0.2,0.5,0.2,8.03
1,"Kylian Mbappé PSG, 23, AM(LR),FW",23(1),2042,10,2,35.3,82.3,0.7,1.2,-,7.89
2,"Erling Haaland Borussia Dortmund, 21, FW",13(1),1145,5,0.9,20.4,69.9,-,-,0.1,7.88
3,"Mohamed Salah Liverpool, 29, AM(CLR),FW",25(1),2229,10,2,33.4,80.2,0.2,0.4,0.3,7.77
4,"Karim Benzema Real Madrid, 34, FW",24,1974,10,1.9,38,85.6,0.1,0.9,0.1,7.71


In [36]:
## TABELAS "PLAYER STATISTICS - PASSING (Home)" ##
player_pas_home = gera_paginas_player_passing(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[4]/div[1]/div[2]/dl[1]/dd[2]/a', qtd_paginas=8)
player_pas_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,"Erling Haaland Borussia Dortmund, 21, FW",7,615,2,1,21.7,69.7,-,-,0.1,8.48
1,"Kylian Mbappé PSG, 23, AM(LR),FW",12,1044,6,2.3,36.3,81.4,0.8,0.9,-,8.29
2,"Dusan Vlahovic Fiorentina, 22, FW",10,898,2,1.3,19.5,72.3,0.3,1,0.3,8.02
3,"Christopher Nkunku RBL, 24, M(CLR),FW",12(1),1081,7,1.9,30.9,82.3,0.5,0.8,0.2,7.93
4,"Robert Lewandowski Bayern, 33, FW",13,1148,1,1.3,20.2,81.3,0.3,0.4,0.2,7.90


In [37]:
## TABELAS "PLAYER STATISTICS - PASSING (Away)" ##
player_pas_away = gera_paginas_player_passing(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[4]/div[1]/div[2]/dl[1]/dd[3]/a', qtd_paginas=8)
player_pas_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,"Robert Lewandowski Bayern, 33, FW",13,1113,-,1.2,22.8,73.3,-,0.6,0.2,8.15
1,"Joshua Kimmich Bayern, 27, D(CR),DMC,M(R)",10,900,6,4,81.2,86.7,3,3.7,0.5,7.97
2,"Mohamed Salah Liverpool, 29, AM(CLR),FW",13,1145,6,2.1,33,79.5,0.2,0.3,0.3,7.75
3,"Phil Foden Man City, 21, AM(CLR),FW",9,783,2,1.8,42.4,90.1,1.7,0.9,0.1,7.71
4,"Zlatan Ibrahimovic AC Milan, 40, FW",6(1),461,1,1.3,27.9,73.8,0.3,0.4,-,7.70


In [38]:
## Opção de filtragem de jogadores - Passing (Remover comentário do código referente à opção de filtro de jogadores desejada e comentar o que não vai usar) ##
## OBS: caso queira obter os dados dos jogadores na guia Passing alterando o filtro 'Default', após executar essa célula, executar as três células acima novamente

# Minumum apps (Default)
btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[4]/div[1]/div[2]/dl[2]/dd[1]/a')

# All players
# btn_filtro = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[4]/div[1]/div[2]/dl[2]/dd[2]/a')

btn_filtro.click()
time.sleep(3)

In [39]:
#### PLAYER - DETAILED ####

# FUNÇÃO PARA GERAR TABELA "DETAILED - PLAYER" ##

# Criando função 'gera_tab_team_detailed' para gerar tabelas com os dados da página na guia 'Detailed'
def gera_tab_player_detailed():
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", dentro da guia "Detailed".'''

    # Criando variáveis para obter os elementos da página
    elemento = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[5]')
    resposta = elemento.get_attribute('innerHTML')
    conteudo_site = BeautifulSoup(resposta, 'html.parser')

    # Selecionando elementos da página para efetuar a raspagem dos dados da página
    elementos_tab_th = conteudo_site.find_all('th') # Nomes das colunas
    elementos_tab_td = conteudo_site.find_all('td') # Dados estatísticos

    # Variável com os elementos para verificar a quantidade de equipes na página
    elementos_tab_dl = conteudo_site.find_all('dl')

    # Criando lista das colunas da tabela 'Detailed'
    colunas_detailed = []

    # 'Looping' para armazenar os nomes das colunas da tabela 'Detailed'
    for elem in elementos_tab_th:
        colunas_detailed.append(elem.text)
    # Renomeando a primeira coluna que será excluída futuramente (antigo nome = 'Player', o mesmo nome da segunda coluna), os dados delas são idênticos ao da segunda,
    # mas com um algarismo numérico junto com o primeiro nome de cada jogador
    colunas_detailed[0] = 'Remover'

    # Criando lista de listas referente aos dados da tabela
    dados_equipes = []
    qtd_colunas = len(colunas_detailed)
    idx = 0 # Índice referente ao primeiro elemento referente aos dados das equipes 'detailed'
    # Quantidade de equipes na página (subtração dos numerais da última e primeira equipes da página localizados no canto inferior direito da página + 1 
    # [... Showing 1 - 20] ...)
    qtd_equipes = int(elementos_tab_dl[9].b.text.split()[6]) - int(elementos_tab_dl[9].b.text.split()[4]) + 1

    # 'Looping' para armazenar os dados das equipes na lista 'dados_equipes'
    for _ in range(qtd_equipes):
        equipe = []
        # 'Looping para buscar os dados de cada um dos equipes disponíveis na página'
        for _ in range(qtd_colunas):
            equipe.append(elementos_tab_td[idx+12].text.strip()) # OBS: os 'elementos_tab_td' com indexes de 0 a 11 são dados que não desejamos na tabela
            idx += 1
        dados_equipes.append(equipe)

    # Criando 'DataFrame' da tabela 'Defensive'
    tab_detailed = pd.DataFrame(dados_equipes, columns=colunas_detailed)

    return tab_detailed.drop(columns='Remover')

# ---------------------------------------------------------------------------------------------------------------------------------------------------------------

## FUNÇÃO PARA GERAR TODAS AS PÁGINAS DA TABELA "DETAILED - PLAYER" ("Overall", "Home" ou "Away") ##

# Criando função 'gera_paginas_player_detailed' para gerar tabelas com os dados da página
def gera_paginas_player_detailed(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[5]/div[1]/div/div[4]/div[1]/dl[1]/dd[1]/a', qtd_paginas=0):
    '''Efetua a raspagem de dados do site "https://1xbet.whoscored.com/Statistics" na página "Player Statistics", todas as páginas dentro da guia "Detailed" de \
um dos seguintes sub-menus: "Overall", "Home" ou "Away". Recebe uma variável do tipo "string" referente ao caminho "xpath" de um dos botões dos sub-menus e uma \
do tipo "int" referente à quantidade de páginas desejadas (valor 0 = todas as páginas).'''

    # Acessando a guia "Detailed"
    btn_detailed = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/ul/li[5]/a')
    btn_detailed.click()
    time.sleep(4)
    btn_submenu = driver.find_element_by_xpath(botao_submenu)
    btn_submenu.click()
    time.sleep(4)

    # Acessando a primeira página da guia
    btn_first = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[5]/div[4]/div/dl[2]/dd[1]/a')
    btn_first.click()
    print('Obtendo os dados da página: 1')
    time.sleep(3)

    # Criando 'DataFrame' inicial referente à tabela
    tabela_player_detailed = gera_tab_player_detailed()

    # Instanciando botão "next" da página 'detailed'
    btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[5]/div[4]/div/dl[2]/dd[3]/a')

    # Criando variáveis para o 'Looping'
    mudar_pagina = ' '.join(btn_next.get_attribute('class').split()) # Validar o funcionamento do 'while'
    contador_pg = 2 # Contador de páginas, à partir da página 2 devido a primeira página já ter sido utilizada para iniciar o 'DataFrame' referente à tabela

    # 'Looping' para gerar tabela com as equipes de todas as páginas da guia 'Detailed'
    while mudar_pagina == 'option clickable':
        btn_next.click()
        print(f'Obtendo os dados da página: {contador_pg}')
        time.sleep(5)
        tabela_temp = gera_tab_player_detailed()
        tabela_player_detailed = pd.concat([tabela_player_detailed, tabela_temp])
        btn_next = driver.find_element_by_xpath('/html/body/div[4]/div[3]/div[5]/div[5]/div[4]/div/dl[2]/dd[3]/a')
        mudar_pagina = ' '.join(btn_next.get_attribute('class').split())
        contador_pg += 1
        if (qtd_paginas > 0) & (contador_pg > qtd_paginas):
            mudar_pagina = 'OK'

    # Reorganizando o index
    tabela_player_detailed.reset_index(drop=True, inplace=True)

    return tabela_player_detailed

In [40]:
## TABELAS "PLAYER STATISTICS - DETAILED (Overall)" ##
player_dtl_overall = gera_paginas_player_detailed(qtd_paginas=8)
player_dtl_overall.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,"Mattia Zaccagni Verona, 26, AM(CL),FW",2,180,2,-,-,2,8.47
1,"Robert Lewandowski Bayern Munich, 33, FW",26,2261,5,0.8,0.8,3.4,8.03
2,"Kylian Mbappé Paris Saint-Germain, 23, AM(LR),FW",24,2042,4,1,0.2,2.8,7.89
3,"Erling Haaland Borussia Dortmund, 21, FW",14,1145,4.1,0.1,0.2,3.8,7.88
4,"Kieran Trippier Newcastle United, 31, D(R),M(R)",4,318,0.5,0.5,-,-,7.87


In [41]:
## TABELAS "PLAYER STATISTICS - DETAILED (Home)" ##
player_dtl_home = gera_paginas_player_detailed(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[5]/div[1]/div/div[4]/div[1]/dl[1]/dd[2]/a', qtd_paginas=8)
player_dtl_home.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,"Erling Haaland Borussia Dortmund, 21, FW",7,615,4.7,0.1,0.1,4.4,8.48
1,"Mattia Zaccagni Verona, 26, AM(CL),FW",2,180,2,-,-,2,8.47
2,"Gerard Moreno Villarreal, 29, AM(CR),FW",6,525,3.5,0.8,0.3,2.3,8.29
3,"Kylian Mbappé Paris Saint-Germain, 23, AM(LR),FW",12,1044,4.3,1,0.1,3.3,8.29
4,"Ferran Torres Manchester City, 22, AM(LR),FW",2,151,3.5,1.5,0.5,1.5,8.13


In [42]:
## TABELAS "PLAYER STATISTICS - DETAILED (Away)" ##
player_dtl_away = gera_paginas_player_detailed(botao_submenu='/html/body/div[4]/div[3]/div[5]/div[5]/div[1]/div/div[4]/div[1]/dl[1]/dd[3]/a', qtd_paginas=8)
player_dtl_away.head()

Obtendo os dados da página: 1
Obtendo os dados da página: 2
Obtendo os dados da página: 3
Obtendo os dados da página: 4
Obtendo os dados da página: 5
Obtendo os dados da página: 6
Obtendo os dados da página: 7
Obtendo os dados da página: 8


,Player,Apps,Mins,Total,OutOfBox,SixYardBox,PenaltyArea,Rating
0,"Andy Delort Montpellier, 30, AM(R),FW",1,90,5,2,1,2,8.24
1,"Robert Lewandowski Bayern Munich, 33, FW",13,1113,4.9,0.8,0.6,3.5,8.15
2,"Joshua Kimmich Bayern Munich, 27, D(CR),DMC,M(R)",10,900,1.1,0.8,-,0.3,7.97
3,"Sérgio Oliveira Roma, 29, DMC",2,159,1,0.5,-,0.5,7.86
4,"Kike Hermoso Real Betis, 22, Defender",1,90,2,-,1,1,7.77
